<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Есть пропуски в столбце "Tenure" около 9%, заменим пропуски медианным значением. 

In [4]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

Уберём столбцы 'RowNumber', 'CustomerId', 'Surname' за ненадобностью и что бы они не влияли на предсказания.

In [5]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Преобразуем категореальные признаки в численные 

In [6]:
data_ohe = pd.get_dummies(data, drop_first=True)

## Исследование задачи

Разделим данные на обучающие и валидационные.

In [7]:
features = data_ohe.drop('Exited', axis=1)
target = data_ohe['Exited']

f, features_valid, t, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)

features_train, features_test, target_train, target_test = train_test_split(f,t, test_size=0.25, random_state=12345)


Обучим модели без учета дисбаланса классов. Напишем функцию для выбора лучше модели.

In [8]:
# Создадим функцию для проверки сразу всех моделей с разными гиперпараметрами которая вернёт нам датафрэйм с колонками 
# 'models','results_f1','result_accuracy' в которых будут лучшие модели и их результаты.
def test_models(f_train, t_train, f_valid, t_valid, class_weight=None):
    depth = 1
    estimators = 1
    
    def metod_list_form(depth, estimators):
        metod_list = [[DecisionTreeClassifier(random_state=12345, class_weight=class_weight, max_depth=depth),20,2],
                      [RandomForestClassifier(random_state=12345,class_weight=class_weight,
                                              max_depth=depth, n_estimators=estimators),50,100],
                      [LogisticRegression(random_state=12345,class_weight=class_weight),2,2]]
        return metod_list
    
    result_df = pd.DataFrame(columns=['models','results_f1','result_accuracy','roc_auc'])
    metod_list = metod_list_form(depth, estimators)
    
    for i in range(len(metod_list)):
        best_model = None
        best_result_f1 = 0
        
        for depth in range(1, metod_list[i][1],5):
            
            for estimators in range(1, metod_list[i][2],5):
                model = metod_list[i][0]
                model.fit(f_train,t_train) 
                predictions = model.predict(f_valid) 
                f1 = f1_score(t_valid, predictions) 
                metod_list = metod_list_form(depth, estimators)
                
                if f1 > best_result_f1:
                    best_model = model
                    best_result_f1 = f1
                    best_result_accuracy = accuracy_score(t_valid, predictions)
                    roc_auc = roc_auc_score(t_valid, model.predict_proba(f_valid)[:,1])
                    
                    
        result_df.loc[len(result_df.index)] = [best_model, best_result_f1, best_result_accuracy,roc_auc] 
    return result_df

In [9]:
best_top_models = test_models(features_train, target_train, features_valid, target_valid)

In [10]:
print(best_top_models['models'][0],
      ':f1 ',best_top_models['results_f1'][0],
      ':accuracy ',best_top_models['result_accuracy'][0], 
      ':roc_auc',best_top_models['roc_auc'][0])

print(best_top_models['models'][1],
      ':f1 ',best_top_models['results_f1'][1],
      ':accuracy ',best_top_models['result_accuracy'][1], 
      ':roc_auc',best_top_models['roc_auc'][1])

print(best_top_models['models'][2],
      ':f1 ',best_top_models['results_f1'][2],
      ':accuracy ',best_top_models['result_accuracy'][2],
      ':roc_auc',best_top_models['roc_auc'][2])

DecisionTreeClassifier(max_depth=6, random_state=12345) :f1  0.5096153846153846 :accuracy  0.847 :roc_auc 0.8415772007426254
RandomForestClassifier(max_depth=31, n_estimators=81, random_state=12345) :f1  0.5824964131994261 :accuracy  0.8545 :roc_auc 0.859376093355229
LogisticRegression(random_state=12345) :f1  0.102880658436214 :accuracy  0.782 :roc_auc 0.6709058452724622


Лучьший результат у модели с параметрами "RandomForestClassifier(max_depth=31, n_estimators=81, random_state=12345)" f1: 0.5824964131994261 accuracy: 0.8545  roc_auc: 0.859376093355229. Даже на валидационной выборки не выдаёт нужный нам результат, нужно улучшать модель.

## Борьба с дисбалансом

Попробуем все три метода борьбы с дисбалансом. 
Для начала опробуем параметр class_weight='balanced'

In [11]:
best_top_balance = test_models(features_train, target_train, features_valid, target_valid, 'balanced')

In [12]:
print(best_top_balance['models'][0],':f1 ',best_top_balance['results_f1'][0],'accuracy ',best_top_balance['result_accuracy'][0])
print(best_top_balance['models'][1],':f1 ',best_top_balance['results_f1'][1],'accuracy ',best_top_balance['result_accuracy'][1])
print(best_top_balance['models'][2],':f1 ',best_top_balance['results_f1'][2],'accuracy ',best_top_balance['result_accuracy'][2])

DecisionTreeClassifier(class_weight='balanced', max_depth=6, random_state=12345) :f1  0.5933852140077821 accuracy  0.791
RandomForestClassifier(class_weight='balanced', max_depth=11, n_estimators=86,
                       random_state=12345) :f1  0.6461916461916462 accuracy  0.856
LogisticRegression(class_weight='balanced', random_state=12345) :f1  0.4550345887778632 accuracy  0.6455


Увеличение выборки.

In [13]:
data.groupby('Exited').count()['Gender'] #Узнаем какой перекос и в какую сторону в нашей выборке

Exited
0    7963
1    2037
Name: Gender, dtype: int64

Отрицательных значений в 4 раза больше положительных.

In [14]:
#Создадим функцияю для увелечения выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [15]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
best_top_up = test_models(features_upsampled, target_upsampled, features_valid, target_valid)

In [16]:
print(best_top_up['models'][0],':f1 ',best_top_up['results_f1'][0],':accuracy ',best_top_up['result_accuracy'][0])
print(best_top_up['models'][1],':f1 ',best_top_up['results_f1'][1],':accuracy ',best_top_up['result_accuracy'][1])
print(best_top_up['models'][2],':f1 ',best_top_up['results_f1'][2],':accuracy ',best_top_up['result_accuracy'][2])

DecisionTreeClassifier(max_depth=6, random_state=12345) :f1  0.5933852140077821 :accuracy  0.791
RandomForestClassifier(max_depth=11, n_estimators=76, random_state=12345) :f1  0.6487695749440715 :accuracy  0.843
LogisticRegression(random_state=12345) :f1  0.4511733535200605 :accuracy  0.6375


Уменьшения выборки.

In [17]:
#Создадим функцияю для уменьшения выборки
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled



In [18]:
features_downsampled, target_downsampled = downsample(features_train,target_train, 0.25)
best_top_down = test_models(features_downsampled, target_downsampled, features_valid, target_valid)

In [19]:
print(best_top_down['models'][0],':f1 ',best_top_down['results_f1'][0],':accuracy ',best_top_down['result_accuracy'][0])
print(best_top_down['models'][1],':f1 ',best_top_down['results_f1'][1],':accuracy ',best_top_down['result_accuracy'][1])
print(best_top_down['models'][2],':f1 ',best_top_down['results_f1'][2],':accuracy ',best_top_down['result_accuracy'][2])

DecisionTreeClassifier(max_depth=6, random_state=12345) :f1  0.5745062836624776 :accuracy  0.763
RandomForestClassifier(max_depth=6, n_estimators=86, random_state=12345) :f1  0.6171003717472119 :accuracy  0.794
LogisticRegression(random_state=12345) :f1  0.44879518072289154 :accuracy  0.634


In [20]:
top_model = (pd.concat([best_top_balance]+[best_top_up]+[best_top_down])
             .sort_values(by=['results_f1'], ascending=False)
            .reset_index(drop=True))
top_model

,models,results_f1,result_accuracy,roc_auc
0,"(DecisionTreeClassifier(max_depth=11, max_feat...",0.648770,0.8430,0.862984
1,"(DecisionTreeClassifier(max_depth=11, max_feat...",0.646192,0.8560,0.862445
2,"(DecisionTreeClassifier(max_depth=6, max_featu...",0.617100,0.7940,0.861498
3,DecisionTreeClassifier(class_weight='balanced'...,0.593385,0.7910,0.819066
4,"DecisionTreeClassifier(max_depth=6, random_sta...",0.593385,0.7910,0.819026
5,"DecisionTreeClassifier(max_depth=6, random_sta...",0.574506,0.7630,0.829786
6,"LogisticRegression(class_weight='balanced', ra...",0.455035,0.6455,0.712009
7,LogisticRegression(random_state=12345),0.451173,0.6375,0.712554
8,LogisticRegression(random_state=12345),0.448795,0.6340,0.712749


## Тестирование модели

Лучше всего себя показал метод борьбы с дисбаланосм по принцыпу увеличения выборки, c параметрами
RandomForestClassifier(max_depth=11, n_estimators=76, random_state=12345). Лучший результат f1: 0.648770

In [21]:
model = top_model['models'][0]
predictions = model.predict(features_test)
print('f1:', f1_score(target_test, predictions), 
      'accuracy:', accuracy_score(target_test, predictions), 
      'roc_auc:', roc_auc_score(target_test, model.predict_proba(features_test)[:,1])) 

f1: 0.5938967136150236 accuracy: 0.827 roc_auc: 0.8459369372090176


Результат тестовой на тестовой выборке нам подходит f1: 0.5938967136150236 accuracy: 0.827 roc_auc: 0.8459369372090176